In [1]:
import unittest
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
import pandas as pd
import logging
from care_logger import (
    init_connection,
    initialize_metadata,
    populate_sample_data,
    add_resident,
    add_care_note,
    analyse_sentiment
)

In [2]:
# Set up logging for tests
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
class TestResidentManagement(unittest.TestCase):
    """
    Test suite for the Resident Management System.
    Uses unittest for structured testing and follows test-driven development principles.
    """
    
    @classmethod
    def setUpClass(cls):
        """
        Set up the test environment before any tests run.
        Creates a test database and populates it with sample data.
        """
        # Create a test database
        cls.engine = create_engine('sqlite:///test_database.db')
        cls.engine, cls.metadata, cls.residents, cls.care_notes = initialize_metadata()
        
        # Populate with sample data
        populate_sample_data()
        logger.info("Test database initialized and populated with sample data")

    def setUp(self):
        """
        Set up before each test method.
        Creates a new database connection for each test.
        """
        self.connection = self.engine.connect()
        logger.info("Created new connection for test")

    def tearDown(self):
        """
        Clean up after each test method.
        Closes the database connection.
        """
        self.connection.close()
        logger.info("Closed test connection")

    def test_basic_data_presence(self):
        """
        Test if the database contains the expected basic data.
        Verifies that both residents and care notes tables have records.
        """
        # Check residents
        resident_count = self.connection.execute(text("SELECT COUNT(*) FROM residents")).scalar()
        self.assertGreater(resident_count, 0, "No residents found in database")
        
        # Check care notes
        notes_count = self.connection.execute(text("SELECT COUNT(*) FROM care_notes")).scalar()
        self.assertGreater(notes_count, 0, "No care notes found in database")
        
        logger.info(f"Found {resident_count} residents and {notes_count} care notes")

    def test_data_quality(self):
        """
        Test the quality and completeness of the data.
        Checks for null values and data integrity.
        """
        # Check for null values in residents
        null_check_residents = """
        SELECT COUNT(*) 
        FROM residents 
        WHERE first_name IS NULL 
        OR last_name IS NULL 
        OR room_num IS NULL
        """
        null_residents = self.connection.execute(text(null_check_residents)).scalar()
        self.assertEqual(null_residents, 0, f"Found {null_residents} residents with null values")
        
        # Check for null values in care notes
        null_check_notes = """
        SELECT COUNT(*) 
        FROM care_notes 
        WHERE note_text IS NULL 
        OR sentiment_score IS NULL 
        OR timestamp IS NULL
        """
        null_notes = self.connection.execute(text(null_check_notes)).scalar()
        self.assertEqual(null_notes, 0, f"Found {null_notes} care notes with null values")
        
        logger.info("Data quality checks passed")

    def test_sentiment_ranges(self):
        """
        Test if sentiment scores are within valid ranges.
        Verifies that all sentiment scores are between -1 and 1.
        """
        query = """
        SELECT 
            MIN(sentiment_score) as min_score,
            MAX(sentiment_score) as max_score,
            AVG(sentiment_score) as avg_score
        FROM care_notes
        """
        stats = self.connection.execute(text(query)).first()
        
        self.assertGreaterEqual(stats.min_score, -1, "Sentiment score below minimum threshold")
        self.assertLessEqual(stats.max_score, 1, "Sentiment score above maximum threshold")
        
        logger.info(f"Sentiment scores range from {stats.min_score:.2f} to {stats.max_score:.2f}")

    def test_resident_care_history(self):
        """
        Test if each resident has a proper care history.
        Ensures no resident is without care notes.
        """
        query = """
        SELECT 
            r.id,
            r.first_name || ' ' || r.last_name as resident_name,
            COUNT(cn.id) as note_count
        FROM residents r
        LEFT JOIN care_notes cn ON r.id = cn.resident_id
        GROUP BY r.id
        HAVING note_count < 1
        """
        residents_without_notes = self.connection.execute(text(query)).fetchall()
        self.assertEqual(
            len(residents_without_notes), 
            0, 
            f"Found {len(residents_without_notes)} residents without care notes"
        )
        
        logger.info("All residents have care history")

    def test_room_assignments(self):
        """
        Test if room assignments are valid.
        Checks for duplicate room assignments and valid room numbers.
        """
        # Check for duplicate room assignments
        query = """
        SELECT 
            room_num,
            COUNT(*) as resident_count
        FROM residents
        GROUP BY room_num
        HAVING resident_count > 1
        """
        duplicate_rooms = self.connection.execute(text(query)).fetchall()
        self.assertEqual(
            len(duplicate_rooms), 
            0, 
            f"Found {len(duplicate_rooms)} rooms with multiple residents"
        )
        
        # Validate room numbers
        room_numbers = self.connection.execute(text("SELECT room_num FROM residents")).fetchall()
        for room in room_numbers:
            self.assertTrue(
                isinstance(room[0], int) or str(room[0]).isdigit(),
                f"Invalid room number found: {room[0]}"
            )
        
        logger.info("All room assignments are valid")

    def test_recent_activity(self):
        """
        Test if there's recent activity in the system.
        Verifies that care notes are being added regularly.
        """
        thirty_days_ago = datetime.now() - timedelta(days=30)
        
        query = """
        SELECT COUNT(*) 
        FROM care_notes
        WHERE timestamp >= :start_date
        """
        
        recent_count = self.connection.execute(
            text(query), 
            {"start_date": thirty_days_ago}
        ).scalar()
        
        self.assertGreater(
            recent_count, 
            0, 
            "No recent care notes found in the last 30 days"
        )
        
        logger.info(f"Found {recent_count} care notes in the last 30 days")

    def test_sentiment_analysis(self):
        """
        Test the sentiment analysis functionality.
        Verifies that the sentiment analyzer produces expected results for sample texts.
        """
        # Test positive sentiment
        positive_text = "Patient had a great day and participated enthusiastically in activities"
        positive_score = analyse_sentiment(positive_text)
        self.assertGreater(positive_score, 0, "Failed to detect positive sentiment")
        
        # Test negative sentiment
        negative_text = "Patient seemed very tired and refused to participate"
        negative_score = analyse_sentiment(negative_text)
        self.assertLess(negative_score, 0, "Failed to detect negative sentiment")
        
        # Test neutral sentiment
        neutral_text = "Patient attended lunch in the dining room"
        neutral_score = analyse_sentiment(neutral_text)
        self.assertTrue(
            -0.1 <= neutral_score <= 0.1, 
            "Failed to identify neutral sentiment"
        )
        
        logger.info("Sentiment analysis tests passed")

In [ ]:
def run_all_tests():
    """
    Run all tests and provide a summary of results.
    """
    # Create a test suite
    suite = unittest.TestLoader().loadTestsFromTestCase(TestResidentManagement)
    
    # Run the tests with a test runner that provides detailed output
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    
    # Print summary
    print("\nTest Summary:")
    print(f"Tests run: {result.testsRun}")
    print(f"Failures: {len(result.failures)}")
    print(f"Errors: {len(result.errors)}")
    
    return result.wasSuccessful()

if __name__ == '__main__':
    success = run_all_tests()
    exit(0 if success else 1)

INFO:care_logger:Database connection successful
INFO:care_logger:Database schema created successfully


INFO:care_logger:Database connection successful
INFO:__main__:Test database initialized and populated with sample data
test_basic_data_presence (__main__.TestResidentManagement.test_basic_data_presence)
Test if the database contains the expected basic data. ... INFO:__main__:Created new connection for test
INFO:__main__:Found 1 residents and 1 care notes
INFO:__main__:Closed test connection
ok
test_data_quality (__main__.TestResidentManagement.test_data_quality)
Test the quality and completeness of the data. ... INFO:__main__:Created new connection for test
INFO:__main__:Data quality checks passed
INFO:__main__:Closed test connection
ok
test_recent_activity (__main__.TestResidentManagement.test_recent_activity)
Test if there's recent activity in the system. ... INFO:__main__:Created new connection for test
INFO:__main__:Found 1 care notes in the last 30 days
INFO:__main__:Closed test connection
ok
test_resident_care_history (__main__.TestResidentManagement.test_resident_care_history)
T


Test Summary:
Tests run: 7
Failures: 0
Errors: 0


: 